# Определение текущего уровня потребительской лояльности.

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.

Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».
Чтобы оценить результаты опроса, оценки обычно делят на три группы:
- 9-10 баллов — «cторонники» (англ. promoters);
- 7-8 баллов — «нейтралы» (англ. passives);
- 0-6 баллов — «критики» (англ. detractors).


Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».

<b>Цель:</b>
- определить текущий уровень потребительской лояльности среди клиентов из России.

<b>Задачи:</b>
1. импортировать необходимые библиотеки,
2. построить дашборд и с его помощью ответить на вопросы:
- Как распределены участники опроса по возрасту, полу и возрасту? 
- Каких пользователей больше: новых или старых? 
- Пользователи из каких городов активнее участвовали в опросе?
- Какие группы пользователей наиболее лояльны к сервису? Какие менее?
- Какой общий NPS среди всех опрошенных?
- Как можно описать клиентов, которые относятся к группе cторонников (англ. promoters)?


<b>Ход работы:</b>
1. [Подключение к базе](#import)
2. ["Исследовательский анализ"](#analysis)
3. Отчет:
- [ссылки на дашборд на сайте Tableau Public](#tableau)
- [ссылка на pdf-файл с презентацией](#presentation)

<a id="import"> </a>
# 1. Подключение к базе

In [1]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db = '/datasets/telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [21]:
query = """
SELECT u.user_id,
       u.lt_day,
       
       CASE 
           WHEN u.lt_day<=365 
               THEN 'True' 
           ELSE 'False' 
       END is_new,
       
       u.age,
       
       CASE 
           WHEN u.gender_segment==1 
               THEN 'женщина' 
           WHEN u.gender_segment==0
               THEN 'мужчина'
       END gender_segment,
       
       u.os_name,
       u.cpe_type_name,
       l.country,
       l.city,
       SUBSTR(age.title, 4) AS age_segment,
       SUBSTR(tr.title, 4) AS traffic_segment,
       SUBSTR(life.title, 4) AS lifetime_segment,
       u.nps_score,
       
       CASE 
           WHEN u.nps_score BETWEEN 9 AND 10
               THEN 'сторонники' 
           WHEN u.nps_score BETWEEN 7 AND 8
               THEN 'нейтралы'
           ELSE 'критики' 
       END nps_group
       
FROM user AS u

LEFT JOIN location AS l ON u.location_id=l.location_id
LEFT JOIN age_segment AS age ON u.age_gr_id=age.age_gr_id
LEFT JOIN traffic_segment AS tr ON u.tr_gr_id=tr.tr_gr_id
LEFT JOIN lifetime_segment AS life on u.lt_gr_id=life.lt_gr_id;
"""

In [22]:
df = pd.read_sql(query, engine)

In [23]:
df.head(5)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,False,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,сторонники
1,A001WF,2344,False,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,сторонники
2,A003Q7,467,False,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,сторонники
3,A004TB,4190,False,44.0,женщина,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,сторонники
4,A004XT,1163,False,24.0,мужчина,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,сторонники


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    502493 non-null  object 
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB


In [11]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

<a id="analysis"> </a>
# 2. "Исследовательский анализ".

In [9]:
df.groupby('age_segment')['age_segment'].count()

age_segment
01 до 16       685
02 16-24     29955
03 25-34    154800
04 35-44    165933
05 45-54     94269
06 55-64     43061
07 66 +      13236
08 n/a         554
Name: age_segment, dtype: int64

In [10]:
df.query('age_segment=="08 n/a"')

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
751,A1E59W,4055,False,NaN,мужчина,ANDROID,SMARTPHONE,Россия,Екатеринбург,08 n/a,04 1-5,08 36+,1,критики
1209,A28ZDT,4243,False,NaN,мужчина,ANDROID,SMARTPHONE,Россия,Хабаровск,08 n/a,04 1-5,08 36+,5,критики
1321,A2GLPQ,2354,False,NaN,мужчина,ANDROID,SMARTPHONE,Россия,Иркутск,08 n/a,06 10-15,08 36+,1,критики
2163,A41C1K,557,False,NaN,мужчина,ANDROID,SMARTPHONE,Россия,РостовнаДону,08 n/a,04 1-5,06 13-24,3,критики
2667,A513VG,4705,False,NaN,мужчина,IOS,SMARTPHONE,Россия,Екатеринбург,08 n/a,23 95-100,08 36+,9,сторонники
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499598,ZUDYU9,520,False,NaN,мужчина,ANDROID,SMARTPHONE,Россия,СанктПетербург,08 n/a,24 100+,06 13-24,9,сторонники
500834,ZWQJRW,220,True,NaN,мужчина,ANDROID,SMARTPHONE,Россия,Самара,08 n/a,03 0.1-1,05 7-12,1,критики
501246,ZXK9AK,3550,False,NaN,мужчина,ANDROID,SMARTPHONE,Россия,Томск,08 n/a,04 1-5,08 36+,4,критики
502376,ZZRS2G,345,True,NaN,мужчина,ANDROID,SMARTPHONE,Россия,Москва,08 n/a,04 1-5,05 7-12,5,критики


In [10]:
df.groupby('gender_segment')['gender_segment'].count()

gender_segment
женщина    272442
мужчина    230051
Name: gender_segment, dtype: int64

In [12]:
df.groupby('is_new')['is_new'].count()

is_new
False    416393
True      86100
Name: is_new, dtype: int64

In [13]:
df.groupby('cpe_type_name')['cpe_type_name'].count()

cpe_type_name
HANDHELD                           1
MIFI ROUTER                        1
MOBILE PHONE/FEATURE PHONE      2471
MOBILE TEST PLATFORM               1
MODEM                              7
NETWORK DEVICE                     3
PHONE                           2531
PORTABLE(INCLUDE PDA)              1
ROUTER                           196
SMARTPHONE                    487959
TABLET                          9274
USB MODEM                         27
WLAN ROUTER                        3
unknown                           18
Name: cpe_type_name, dtype: int64

# 3. Отчет.

<a id="tableau"> </a>
### Укажите ссылку на дашборд на сайте Tableau Public:

<https://public.tableau.com/app/profile/anasatasia3818/viz/ya_project_2_loyal/ratio?publish=yes>

<a id="presentation"> </a>
### Укажите ссылку на pdf-файл с презентацией:

<https://github.com/ziadove/ya_project_6_communication/blob/d49afa38b1ca7fed7f8ba5b205105d5f95ed3821/presentation.pdf>